In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [4]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: float # strike rate
    bpb: float # balls per boundary
    boundary_percentage: float
    summary: str

In [9]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    sr = (state['runs'] / state['balls']) * 100 if state['balls'] > 0 else 0
    return {'sr':sr}

def calculate_bpb(state: BatsmanState) -> BatsmanState:
    total_boundaries = state['fours'] + state['sixes']
    bpb = (state['balls'] / total_boundaries) if total_boundaries > 0 else float('inf')
    return {'bpb':bpb}

def calculate_boundary_percentage(state: BatsmanState) -> BatsmanState:
    total_boundaries = state['fours'] + state['sixes']
    boundary_percentage = (total_boundaries / state['balls']) * 100 if state['balls'] > 0 else 0
    return {'boundary_percentage':boundary_percentage}

def summary(state: BatsmanState) -> str:
    summary= (f"Runs: {state['runs']}, Balls: {state['balls']}, Fours: {state['fours']}, "
            f"Sixes: {state['sixes']}, Strike Rate: {state['sr']:.2f}, "
            f"Balls per Boundary: {state['bpb']:.2f}, Boundary Percentage: {state['boundary_percentage']:.2f}%")
    return {'summary':summary}

In [10]:
graph = StateGraph(BatsmanState)
graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_boundary_percentage)
graph.add_node('summary', summary)

graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percentage')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percentage', 'summary')

graph.add_edge('summary', END)

workflow = graph.compile()

In [11]:
initial_state={
    'runs': 85,
    'balls': 60,
    'fours': 8,
    'sixes': 3
}

output_state = workflow.invoke(initial_state)
print(output_state)

{'runs': 85, 'balls': 60, 'fours': 8, 'sixes': 3, 'sr': 141.66666666666669, 'bpb': 5.454545454545454, 'boundary_percentage': 18.333333333333332, 'summary': 'Runs: 85, Balls: 60, Fours: 8, Sixes: 3, Strike Rate: 141.67, Balls per Boundary: 5.45, Boundary Percentage: 18.33%'}
